# Quantum benchmark

## I - Introduction

## II - Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_score, KFold
#Import classical libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt

plt.style.use('ggplot')

import functools

from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.datasets import ad_hoc_data
import logging

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer

from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal, ZZFeatureMap
import qiskit

In [2]:
import warnings
warnings.filterwarnings('ignore')

## III - Data 

In [3]:
# Read out CSV

df = pd.read_csv('UCI_Credit_Card.csv', sep=',')
df = df.sample(1400)

In [4]:
%%script false --no-raise-error

import sweetviz as sv

#EDA using Autoviz
sweet_report = sv.analyze(df)

#Saving results to HTML file
sweet_report.show_html('sweet_report.html')

## IV - Modelisation

### Classical

In [5]:
df_labels = df['default.payment.next.month']
df.drop(['default.payment.next.month'],axis = 1,inplace = True)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df, df_labels, test_size=0.2, random_state=42)

## Quantum Approaches

In [7]:
np.unique(df_labels)

array([0, 1])

In [8]:
from sklearn.decomposition import NMF

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
# Standardizing the features
minmax = MinMaxScaler()
minmax.fit(X_train, y_train)
X_train = minmax.transform(X_train)
X_test = minmax.transform(X_test)

In [11]:
nmf = NMF(n_components=2, max_iter=5000)

In [12]:
nmf.fit(X_train, y_train)

NMF(max_iter=5000, n_components=2)

In [15]:
df_nmf = nmf.transform(X_train)


In [16]:
nmf_df = pd.DataFrame(data = df_nmf
             , columns = ['principal component 1', 'principal component 2'])



In [17]:
n_dim = len(nmf_df.columns)

## Split train test 

In [18]:
# Split dataset into train and test

sample_train, sample_test, label_train, label_test = train_test_split(
     nmf_df, y_train, test_size=0.2, random_state=22)


sample_train = sample_train.to_numpy()
sample_test = sample_test.to_numpy()
# Normalize

'''std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)

# Scale for better fit within the feature map

samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)

# Select a sample for a better control of the research and wall time
'''
train_size = 800#160
sample_train = sample_train[:train_size]
label_train = label_train[:train_size]

test_size = 200 #40
sample_test = sample_test[:test_size]
label_test = label_test[:test_size]

In [19]:
# Basic parameters for hybrid model

seed = 8500
feature_dim = n_dim
num_reps = 2
num_shots =256 


## Hybrid

In [20]:
# Define feature_map

feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=num_reps)

# Define the backend
backend = QuantumInstance(
    BasicAer.get_backend("qasm_simulator"), shots=num_shots, seed_simulator=seed, seed_transpiler=seed
)

# Define the kernel

kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

# Model run
svc = SVC(kernel=kernel.evaluate)
#svc.fit(sample_train, label_train)
#score = svc.score(sample_test, label_test)

#print(f"Callable kernel classification test score: {score}")

In [21]:
#result_predict = svc.predict(sample_test)

In [22]:
#print(metrics.classification_report(label_test,result_predict))

In [23]:
from sklearn.model_selection import cross_validate
from tqdm import tqdm
def evaluate_ml_model(_models, X, y, n_fold=10, metric='precision'):
    ''' Function to evaluate a ML and QML model with a list of metrics
    
    
    '''
    results = pd.DataFrame()
    kfold = KFold(n_splits=n_fold)
    columns = []
    for name, model in tqdm(_models):
        # -------------------
        # Variables initialization 
        _df = pd.DataFrame()
        names = []
        means = []
        stds = []
        
        # -------------------
        # k-fold Cross validation
        cv_results = cross_validate(model, X, y, cv=kfold, scoring=metric)
        
        # -------------------
        # Compute the mean and standard deviation 
        for _name, _array in cv_results.items():
            names.append(_name)
            means.append(round(100*_array.mean(), 2))
            stds.append(round(100*_array.std(), 2))
        # -------------------
        # Save the results in a dataframe 
        _df =  pd.DataFrame([means, stds], columns=names)
        columns.extend([name+' mean (%)', name+' std (%)'])
        #results = results.join(_df, on=_df.index)
        results = results.append(_df)
    results.index = columns
    print(results)
    return results

In [24]:
models = []
models.append(('LR', LogisticRegression(max_iter=1000)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('qsvc', svc))
_metrics = ['precision', 'recall', 'f1', 'accuracy',  'matthews_corrcoef','balanced_accuracy']

In [25]:
df_results = pd.DataFrame()
df_results = evaluate_ml_model(models, sample_train, label_train, n_fold=10, metric=_metrics)
df_results 

100%|██████████████████████████████████████████| 6/6 [1:17:54<00:00, 779.16s/it]

               fit_time  score_time  test_precision  test_recall  test_f1  \
LR mean (%)        0.36        0.33            0.00         0.00     0.00   
LR std (%)         0.31        0.34            0.00         0.00     0.00   
KNN mean (%)       0.04        0.21           27.16         8.11    12.13   
KNN std (%)        0.01        0.01           18.23         4.56     6.75   
CART mean (%)      0.22        0.24           27.02        25.80    25.54   
CART std (%)       0.12        0.21           11.57         6.59     6.64   
NB mean (%)        0.05        0.17           40.00         3.70     6.69   
NB std (%)         0.04        0.12           37.42         3.31     5.94   
SVM mean (%)       0.78        0.26            0.00         0.00     0.00   
SVM std (%)        0.13        0.01            0.00         0.00     0.00   
qsvc mean (%)  40007.00     6739.19            0.00         0.00     0.00   
qsvc std (%)    4235.31      520.78            0.00         0.00     0.00   

,fit_time,score_time,test_precision,test_recall,test_f1,test_accuracy,test_matthews_corrcoef,test_balanced_accuracy
LR mean (%),0.36,0.33,0.00,0.00,0.00,76.50,0.00,50.00
LR std (%),0.31,0.34,0.00,0.00,0.00,5.15,0.00,0.00
KNN mean (%),0.04,0.21,27.16,8.11,12.13,72.75,1.68,50.32
KNN std (%),0.01,0.01,18.23,4.56,6.75,4.14,9.24,2.84
CART mean (%),0.22,0.24,27.02,25.80,25.54,65.38,3.88,51.80
CART std (%),0.12,0.21,11.57,6.59,6.64,3.99,9.35,4.40
NB mean (%),0.05,0.17,40.00,3.70,6.69,75.87,5.89,50.89
NB std (%),0.04,0.12,37.42,3.31,5.94,4.58,11.87,2.28
SVM mean (%),0.78,0.26,0.00,0.00,0.00,76.50,0.00,50.00
SVM std (%),0.13,0.01,0.00,0.00,0.00,5.15,0.00,0.00


## Pennylane

In [26]:
from pennylane import numpy as np

In [27]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [28]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

#print(weights_init, bias_init)

In [29]:
circuit(weights_init, sample_train[0])

tensor(-0.00921692, requires_grad=True)

In [30]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [31]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [32]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [33]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [34]:
Y = np.array(label_train * 2 - np.ones(len(label_train)),requires_grad=True)  # shift label from {0, 1} to {-1, 1}
X = np.array(sample_train, requires_grad=True)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(0.06278958, requires_grad=True), tensor(0.06542052, requires_grad=True)], Y = -1
X = [tensor(0.00110415, requires_grad=True), tensor(0.13349335, requires_grad=True)], Y = -1
X = [tensor(0.06062075, requires_grad=True), tensor(0.1244901, requires_grad=True)], Y = -1
X = [tensor(0.06143445, requires_grad=True), tensor(0.08450835, requires_grad=True)], Y =  1
X = [tensor(0.00035445, requires_grad=True), tensor(0.18628152, requires_grad=True)], Y = -1


In [35]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [36]:
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(20):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    '''if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )'''
    if metrics.f1_score(Y, predictions, average='binary', pos_label=1) > abest:
        wbest = weights
        bbest = bias
        abest = metrics.f1_score(Y, predictions, average='binary')
        print('New best')
    prec = metrics.f1_score(Y, predictions, average='binary')
    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), prec
        )
    )
    
    

Iter:     1 | Cost: 0.7607421 | Accuracy: 0.0000000 
Iter:     2 | Cost: 0.7271964 | Accuracy: 0.0000000 
Iter:     3 | Cost: 0.7413049 | Accuracy: 0.0000000 
Iter:     4 | Cost: 0.7211073 | Accuracy: 0.0000000 
Iter:     5 | Cost: 0.7329981 | Accuracy: 0.0000000 
Iter:     6 | Cost: 0.9460526 | Accuracy: 0.0000000 
Iter:     7 | Cost: 0.9878273 | Accuracy: 0.0000000 
Iter:     8 | Cost: 0.9201012 | Accuracy: 0.0000000 
Iter:     9 | Cost: 0.7791576 | Accuracy: 0.0000000 
Iter:    10 | Cost: 0.7169658 | Accuracy: 0.0000000 
Iter:    11 | Cost: 0.7397387 | Accuracy: 0.0000000 
Iter:    12 | Cost: 0.8366338 | Accuracy: 0.0000000 
Iter:    13 | Cost: 0.8008213 | Accuracy: 0.0000000 
Iter:    14 | Cost: 0.7423685 | Accuracy: 0.0000000 
Iter:    15 | Cost: 0.7198289 | Accuracy: 0.0000000 
Iter:    16 | Cost: 0.7308869 | Accuracy: 0.0000000 
Iter:    17 | Cost: 0.7249609 | Accuracy: 0.0000000 
Iter:    18 | Cost: 0.7247584 | Accuracy: 0.0000000 
Iter:    19 | Cost: 0.7232587 | Accuracy: 0.00

In [37]:
Yte = np.array(label_test * 2 - np.ones(len(label_test)))
Xte = np.array(normalize(sample_test))

In [38]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

IndexError: tuple index out of range

In [ ]:
print(metrics.classification_report(predictions,Yte))

In [ ]:
print(f'''

    Precision: {round(100*metrics.precision_score(predictions,Yte),2)}%
    Recall: {round(100*metrics.recall_score(predictions,Yte),2)}%
    f1: {round(100*metrics.f1_score(predictions,Yte),2)}%
    Accuracy: {round(100*metrics.accuracy_score(predictions,Yte),2)}%
    Balanced accuracy: {round(100*metrics.balanced_accuracy_score(predictions,Yte),2)}%
    Matthew corcorref: {round(100*metrics.matthews_corrcoef(predictions,Yte),2)}%
    ''')